## MAIN PROGRAM FILE

### Import dependencies

In [1]:
import pandas as pd
import os
from config import gkey
import gmaps
from scipy.stats import linregress
from matplotlib import pyplot as plt
from scipy import stats
import numpy as np
import requests
from pprint import pprint
import json

#### Calling the CSV files and combining the data on both CSVs for State = Delaware

In [2]:
###CALLLING OUT CSV FILE LOCATIONS
csv_path_1 = os.path.join ("..","Resources","Census_Data_2015.csv")
csv_path_2 = os.path.join("..", "Resources","Tract_lat_lon.csv")

### READING CSV DATA INTO DATA FRAMES
Maindata_1_df = pd.read_csv(csv_path_1)
lat_lon_tract_df = pd.read_csv(csv_path_2)

###CHECKING DATA FRAME HEADS TO ENSURE DATA IS BEING CORRECTLY READ
Maindata_1_df.head()
lat_lon_tract_df.head()

###MERGING TWO DATA FRAMES TO GET FINAL DATAFRAME WITH CENSUS AND GEOGROPHIC INFO COMBINED
# THE INNER MERGE WILL ONLY SHOW THOSE ROWS THAT HAVE INFO IN BOTH CSV FILES. IN DOING THIS WE LOST DATA OF 25 TRACT WHICH DID NOT HAVE LAT LONGS

Combined_df = pd.merge(Maindata_1_df, lat_lon_tract_df, on='CENSUSTRACT', how = 'inner')

###CHECKING DATA FRAME SHAPE TO SEE HOW MANY ROWS AND COLUMNS ARE THERE. 
###THIS HV_final_df IS THE DATA FRAME THAT CAN BE USED BY ALL TEAM MEMBERS FOR THEIR INDIVIDUAL PROCESSING


###OUTPUT THE FINAL DATA FRAME WITH LAT LON + CENSUS DATA INTO CSV FILE IN OUTPUT FOLDER
###HV_final_df.to_csv(os.path.join("..","Output", "Census_data_and_lat_lon.csv"), encoding = "utf-8", index = False)
Combined_df.head()

## DATA FRAME FOR ALL DATA FOR A SINGLE STATE TO REDUCE COMPUTATIONAL TIME FOR ETL PROJECT##
## SELECTED BELOW IS DATA FOR DELAWARE  ##


Combined_State_df = Combined_df[Combined_df ["State"] == 'Delaware']
Combined_State_df.head()


,CENSUSTRACT,State,County,Urban,POP2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractOMultir,TractHispanic,TractHUNV,TractSNAP,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,LAT,LON
13674,10001040100,Delaware,Kent,0,6541,2325,0,0,0.000000,0,...,183,247,39,418,124745855.0,0,48.165,0.000,39.237284,-75.694741
13675,10001040201,Delaware,Kent,0,5041,1849,0,2,0.000397,0,...,290,312,160,322,9730214.0,37235,3.757,0.014,39.290841,-75.637508
13676,10001040202,Delaware,Kent,1,12763,4451,0,340,0.026639,0,...,568,593,121,249,31927916.0,700605,12.327,0.271,39.263964,-75.611123
13677,10001040203,Delaware,Kent,0,5017,1877,0,0,0.000000,0,...,269,261,146,289,59860927.0,1054612,23.112,0.407,39.285868,-75.550836
13678,10001040501,Delaware,Kent,1,4923,1910,0,0,0.000000,0,...,392,413,142,345,10394085.0,0,4.013,0.000,39.199584,-75.543902


#### Copied the tables to a new table with the columns we need

In [3]:
sv_new_Combined_State_df = Combined_State_df[['CENSUSTRACT', 'State', 'County', 'POP2010', 'OHU2010', 'MedianFamilyIncome', 'TractHUNV', 'PovertyRate', 'LAT', 'LON']].copy()
sv_new_Combined_State_df.head()

,CENSUSTRACT,State,County,POP2010,OHU2010,MedianFamilyIncome,TractHUNV,PovertyRate,LAT,LON
13674,10001040100,Delaware,Kent,6541,2325,71188,39,10.7,39.237284,-75.694741
13675,10001040201,Delaware,Kent,5041,1849,54826,160,8.7,39.290841,-75.637508
13676,10001040202,Delaware,Kent,12763,4451,73155,121,5.3,39.263964,-75.611123
13677,10001040203,Delaware,Kent,5017,1877,69273,146,11.9,39.285868,-75.550836
13678,10001040501,Delaware,Kent,4923,1910,57891,142,16.9,39.199584,-75.543902


#### Rename the columns

In [4]:
columns_rename = ['Census_tract', 'State', 'County', 'Total_Population', 'Total_Housing_Units', 'Median_Family_Income', 'Units_Without_Vehicle', 'Poverty_Rate',  'Latitude', 'Longitude' ]

In [5]:
sv_new_Combined_State_df.columns=columns_rename

In [6]:
sv_new_Combined_State_df.head()

,Census_tract,State,County,Total_Population,Total_Housing_Units,Median_Family_Income,Units_Without_Vehicle,Poverty_Rate,Latitude,Longitude
13674,10001040100,Delaware,Kent,6541,2325,71188,39,10.7,39.237284,-75.694741
13675,10001040201,Delaware,Kent,5041,1849,54826,160,8.7,39.290841,-75.637508
13676,10001040202,Delaware,Kent,12763,4451,73155,121,5.3,39.263964,-75.611123
13677,10001040203,Delaware,Kent,5017,1877,69273,146,11.9,39.285868,-75.550836
13678,10001040501,Delaware,Kent,4923,1910,57891,142,16.9,39.199584,-75.543902


#### Calling on google places API for getting car dealer information and combining the date received with the above table 

In [7]:
params = {
    "radius": 50000,
    "types": "car_dealer",
    "key": gkey
}

# USING LAT AND LON FROM THE STATE DATA FRAME ABOVE#
for index, row in sv_new_Combined_State_df.iterrows():
    
    # GET LAT & LON FROM COMBINED_TEXAS_DF
    lat = row["Latitude"]
    lng = row["Longitude"]
   
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# make request and print url
dealers_info = requests.get(base_url, params=params)


    # convert to json
dealersinfo_response = dealers_info.json()

  
pprint(dealersinfo_response)


{'html_attributions': [],
 'next_page_token': 'CtQCRgEAADx9Wd07WJ-_19InMiXWQRZ4UmV6UAxgb6RA0udhgdBA_Db_bJ7j9M-QyFuxeF3Z_Dijjujhh7in2Gbl6RLTOMHXWDLSEQNWi42zK61wUmQDsEB--qY7UeJuhMWfY6aBNY1nRs2y7x2ZIguJ7v1c6VJFVlaOvM3mfs9LVJZHMCpTKEj4r2iX8ICVhLFCqKKPw9lt3kl5Gi1DOs6mTFBB1hzXAV8gNGzg1utIFpOZsAB5NxKPGOu1t1I3kzfKoPn8L5_3GIaEIn6Gw3uZBJ9N6blxG69mKk6CvB1-CeamBhNjPWbFq-mjy5A2QVePpKzZtVRGZE26RLqoMQU_qHX8RD6sIZM2XcX6xdNsyejcOCU_ZvQ84TsbK9UMxuJ_ewDx35B8nSd0XgR2f_EsUssgmrP5C-6__YKh934Ri6CtejedtPIU96IgGDkZXEK0vO_1vRIQIcniymwdupjUZU5sSeWO-xoUiovQ4C98EMAuTbv9V0CciK79gq8',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 39.1526773, 'lng': -75.5128244},
                           'viewport': {'northeast': {'lat': 39.1539336302915,
                                                      'lng': -75.5115231697085},
                                        'southwest': {'lat': 39.15123566970851,
                                                      'lng': -75.514221

In [8]:
## GOOGLE PLACES API CALL OUT FOR GETTING DEALER INFO

car_dealers = []

# PARAMS DICT TO UPDATE ALONG EACH ROW
params = {
    "radius": 50000,
    "types": "car_dealer",
    "key": gkey
}

# USING LAT AND LON FROM THE STATE DATA FRAME ABOVE#
for index, row in sv_new_Combined_State_df.iterrows():
    
    # GET LAT & LON FROM COMBINED_TEXAS_DF
    lat = row["Latitude"]
    lng = row["Longitude"]
   
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Delaware" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    dealers_info = requests.get(base_url, params=params)


    # convert to json
    dealers_response = dealers_info.json()
  
    try:
        car_dealers.append(dealers_response["results"][1]["name"])
    except:
        car_dealers.append("Car dealer not found")

# Dataframe with nearest car dealer
sv_new_Combined_State_df["Car Dealer"] = car_dealers
sv_new_Combined_State_df.head(100)
    
      

,Census_tract,State,County,Total_Population,Total_Housing_Units,Median_Family_Income,Units_Without_Vehicle,Poverty_Rate,Latitude,Longitude,Car Dealer
13674,10001040100,Delaware,Kent,6541,2325,71188,39,10.7,39.237284,-75.694741,Carman Chrysler Jeep Dodge
13675,10001040201,Delaware,Kent,5041,1849,54826,160,8.7,39.290841,-75.637508,Winner Ford
13676,10001040202,Delaware,Kent,12763,4451,73155,121,5.3,39.263964,-75.611123,Winner Ford
13677,10001040203,Delaware,Kent,5017,1877,69273,146,11.9,39.285868,-75.550836,Martin Dealerships
13678,10001040501,Delaware,Kent,4923,1910,57891,142,16.9,39.199584,-75.543902,Winner Ford
...,...,...,...,...,...,...,...,...,...,...,...
13769,10003013501,Delaware,New Castle,6602,2452,128802,118,1.4,39.781704,-75.658343,Martin Dealerships
13770,10003013503,Delaware,New Castle,7472,2620,117326,80,3.8,39.781677,-75.693759,"Pacifico Ford, Inc."
13771,10003013505,Delaware,New Castle,3147,1128,168553,27,2.1,39.765072,-75.718149,Colonial Hyundai
13772,10003013506,Delaware,New Castle,4833,1617,155625,0,0.6,39.752546,-75.734866,Colonial Hyundai


In [ ]:
# Python program to convert 
# JSON file to CSV 
  
  
import csv 
  
  
# Opening JSON file and loading the data 
# into the variable data 
with open('dealers_info.json') as json_file: 
    data = json.load(json_file) 
  
car_dealers_info = data['emp_details'] 
  
# now we will open a file for writing 
data_file = open('data_file.csv', 'w') 
  
# create the csv writer object 
csv_writer = csv.writer(data_file) 
  
# Counter variable used for writing  
# headers to the CSV file 
count = 0
  
for emp in employee_data: 
    if count == 0: 
  
        # Writing headers of CSV file 
        header = emp.keys() 
        csv_writer.writerow(header) 
        count += 1
  
    # Writing data of CSV file 
    csv_writer.writerow(emp.values()) 
  
data_file.close() 